In [1]:
import requests
import pandas as pd
import numpy as np
import json
import time

In [8]:
apiKey = "IaF6zOUaj8VG4ambmUwOodEFaxBnrR3A"
apiUrl = "https://aeroapi.flightaware.com/aeroapi/"
auth_header = {'x-apikey':apiKey}

# GET flights from DMK airport

In [9]:
Airport_id = 'DMK'
Airport_start = '2022-12-01T00:00:00Z'
Airport_end = '2022-12-04T00:00:00Z'

# Y-M-D
# 1) 2022-12-01 - 2022-12-04


In [10]:
# #----- get flight data from DMK airport -----#

response = requests.get(apiUrl + f"airports/{Airport_id}/flights?start=2022-12-26T08%3A00%3A00Z&end=2022-12-26T11%3A00%3A00Z",
 headers=auth_header)

# if response.status_code == 200:
#     print(response.json())
# else:
#     print("Error executing request")

In [11]:
print(json.dumps(response.json(), indent=4))

{
    "arrivals": [
        {
            "ident": "NOK505",
            "ident_icao": "NOK505",
            "ident_iata": "DD505",
            "fa_flight_id": "NOK505-1671875100-schedule-0759",
            "operator": "NOK",
            "operator_icao": "NOK",
            "operator_iata": "DD",
            "flight_number": "505",
            "registration": "HS-DBS",
            "atc_ident": null,
            "inbound_fa_flight_id": "NOK504-1671867420-schedule-0627",
            "codeshares": [],
            "codeshares_iata": [],
            "blocked": false,
            "diverted": false,
            "cancelled": false,
            "position_only": false,
            "origin": {
                "code": "VTSS",
                "code_icao": "VTSS",
                "code_iata": "HDY",
                "code_lid": null,
                "timezone": "Asia/Bangkok",
                "name": "Hat Yai Int'l",
                "city": "Hat Yai / Songkhla",
                "airport_info_url": "/a

In [15]:
Arrivals_FlightID = []
Departures_FlightID = []

In [1]:
# tlm_flight = [
#     'TLM100-1671584040-schedule-0562',
#     'TLM100-1671497100-schedule-0570',
#     'TLM100-1671929100-schedule-0825'
# ]
# Departures_FlightID = tlm_flight

In [16]:
#----- add flight data from DMK airport -----#
for i in response.json()['arrivals']:
    if 'schedule' in i['fa_flight_id']:
        Arrivals_FlightID.append(i['fa_flight_id'])
        
for i in response.json()['departures']:
    if 'schedule' in i['fa_flight_id']:
        Departures_FlightID.append(i['fa_flight_id'])

In [17]:
for item in Departures_FlightID:
    print(item)

NOK578-1671879840-schedule-0913
AIQ3223-1671878940-schedule-0501
AIQ3245-1671877800-schedule-0571
AIQ3439-1671878940-schedule-0490
NOK328-1671877800-schedule-0561
NOK348-1671877800-schedule-0602
NOK532-1671876960-schedule-0482
AIQ3306-1671876960-schedule-0494
NOK134-1671876000-schedule-0634
AIQ638-1671875100-schedule-0774
AIQ351-1671875100-schedule-0790
NOK176-1671875100-schedule-0757


# GET data from flight ID

In [19]:
#----- get data from flight ID -----#
Dict_flights = {}

i = 1
for id in Departures_FlightID:
    response = requests.get(apiUrl + f"flights/{id}/track",
    headers=auth_header)

    if response.status_code == 200:
        print(i)
        #----- Add data to dict -----#
        Dict_flights[id] = pd.DataFrame.from_dict(response.json()['positions'])
        
        #----- check time -----#
        Dict_flights[id] = Dict_flights[id].drop_duplicates(subset=['timestamp'], keep='first')

        #----- altitude <= 100k ft. & index[1]-index[0]==1 -----#
        Dict_flights[id] = Dict_flights[id][Dict_flights[id]['altitude']<=100]
        Dict_flights[id]['check_index_1'] = Dict_flights[id].index
        Dict_flights[id]['check_index_2'] = Dict_flights[id]['check_index_1'].shift(-1)
        Dict_flights[id]['drop'] = Dict_flights[id]['check_index_2']-Dict_flights[id]['check_index_1'] == 1
            
        #----- loop append only Departures -----#
        res = pd.DataFrame()
        n = 0
        while(Dict_flights[id].iloc[n]['drop']) :
            res = res.append(Dict_flights[id].iloc[n])
            # res = pd.concat([res, Dict_flights[id].iloc[n]])
            n+=1
        res = res.append(Dict_flights[id].iloc[n])
        # res = pd.concat([res, Dict_flights[id].iloc[n]])
        
        #----- interpolate 1 sec -----#
        res['timestamp'] = pd.to_datetime(res.timestamp)
        nidx = np.arange(res['timestamp'][0], res['timestamp'].iloc[-1], 1000000 )
        nidx = pd.to_datetime(nidx)

        res['timestamp'] = res['timestamp'].round('S').dt.tz_localize(None)
        res.set_index('timestamp', inplace=True)
        res = res.reindex(res.index.union(nidx))

        #----- interpolate ['altitude', 'groundspeed', 'heading', 'latitude', 'longitude'] -----#
        res = res[['fa_flight_id', 'altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
        res['altitude'] = pd.to_numeric(res['altitude'])
        res['groundspeed'] = pd.to_numeric(res['groundspeed'])
        res['heading'] = pd.to_numeric(res['heading'])
        res = res.interpolate(method='time',limit_direction='both',limit=100)
        res['fa_flight_id'] = id
        
        #----- date-time index to timestamp -----#
        res.reset_index(inplace=True)
        res.rename(columns={'index':'timestamp'},inplace=True)
        
        #----- ONLY TEST -----#
        res.to_csv(f'ONLY_TEST/{id}.csv')
            
    else:
        print("Error executing request")
        
    i+=1
    time.sleep(10)

1


C:\Users\Nopnpr\AppData\Local\Temp\ipykernel_21764\1999800881.py:35: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  nidx = np.arange(res['timestamp'][0], res['timestamp'].iloc[-1], 1000000 )


2
3
4
5
6
7
8
9
10
11
12


In [55]:
# Dict_flights['SNY20-1670370011-adhoc-0'].loc[:, ~Dict_flights['SNY20-1670370011-adhoc-0'].index.duplicated()]

In [56]:
# Dict_flights.keys()

In [57]:
# print(json.dumps(response.json(), indent=4))

In [58]:
# df = pd.read_csv(f'flights_data/{id}.csv')
# df.head()

# test ไม่ใช้จ้า

In [59]:
# Dict_flights = {}

# Dict_flights['AIQ642-1669939380-schedule-0476'] = pd.read_csv(f'flights_data/AIQ642-1669939380-schedule-0476.csv')
# Dict_flights['AIQ642-1669939380-schedule-0476']['fa_flight_id'] = 1

# Dict_flights['NOK340-1669938420-schedule-0407'] = pd.read_csv(f'flights_data/NOK340-1669938420-schedule-0407.csv')
# Dict_flights['NOK340-1669938420-schedule-0407']['fa_flight_id'] = 2

# Dict_flights['AIQ642-1669939380-schedule-0476'].head()

In [60]:
# Dict_flights['AIQ642-1669939380-schedule-0476']

# Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = pd.to_datetime(Dict_flights['AIQ642-1669939380-schedule-0476'].timestamp)

# nidx = np.arange(Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'][0], Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].iloc[-1], 1000000 )
# nidx = pd.to_datetime(nidx)
# # nidx

# Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'] = Dict_flights['AIQ642-1669939380-schedule-0476']['timestamp'].round('S').dt.tz_localize(None)
# Dict_flights['AIQ642-1669939380-schedule-0476'].set_index('timestamp', inplace=True)
# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].reindex(Dict_flights['AIQ642-1669939380-schedule-0476'].index.union(nidx))

# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'][['altitude', 'groundspeed', 'heading', 'latitude', 'longitude']]
# Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['altitude'])
# Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['groundspeed'])
# Dict_flights['AIQ642-1669939380-schedule-0476']['heading'] = pd.to_numeric(Dict_flights['AIQ642-1669939380-schedule-0476']['heading'])
# Dict_flights['AIQ642-1669939380-schedule-0476'] = Dict_flights['AIQ642-1669939380-schedule-0476'].interpolate(method='time',limit_direction='both',limit=50)

# Dict_flights['AIQ642-1669939380-schedule-0476']

In [61]:
Dict_flights.keys()

dict_keys(['AIQ3362-1670716320-schedule-0617', 'NOK340-1670716320-schedule-0616', 'NOK122-1670716320-schedule-0607', 'AIQ3437-1670716320-schedule-0611', 'AIQ311-1670716320-schedule-0614', 'AIQ3188-1670716320-schedule-0603', 'AIQ512-1670716320-schedule-0609', 'AIQ3029-1670715240-schedule-0553', 'TLM702-1670715240-schedule-0537', 'AIQ3102-1670715240-schedule-0556', 'NOK100-1670714280-schedule-0369', 'NOK570-1670714280-schedule-0361', 'AIQ3252-1670714280-schedule-0359'])

In [62]:
# Dict_flights['SNY20-1670370011-adhoc-0'].reset_index(inplace=True)
# Dict_flights['SNY20-1670370011-adhoc-0']

In [63]:
Dict_flights['SNY20-1670370011-adhoc-0'] = Dict_flights['SNY20-1670370011-adhoc-0'].drop_duplicates(subset=['timestamp'], keep='first')

KeyError: 'SNY20-1670370011-adhoc-0'

In [ ]:
Dict_flights['SNY20-1670370011-adhoc-0'].reset_index(inplace=True)

In [ ]:
Dict_flights['SNY20-1670370011-adhoc-0'].rename(columns={'index':'timestamp'},inplace=True)

In [ ]:
# Dict_flights['NOK600-1670198400-schedule-0700'].between_time('23:00', '07:00')
if Dict_flights['NOK600-1670198400-schedule-0700'].iloc[:1].between_time('23:00', '07:00').empty :
    print(1)
else:
    print(0)
# Dict_flights['NOK600-1670198400-schedule-0700']

0


In [ ]:
test_dict = {
        'day':{},
        'night':{}
    }

In [ ]:
test_dict['night']['1'] = Dict_flights['SNY20-1670370011-adhoc-0'].copy()

In [ ]:
test_dict['night']['1'][test_dict['night']['1']['altitude'] <= 50]

,timestamp,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
0,2022-12-06 23:49:06,SNY20-1670370011-adhoc-0,3.0000,81.0000,209.0000,13.905900,100.604990
1,2022-12-06 23:49:07,SNY20-1670370011-adhoc-0,3.0625,81.6250,208.9375,13.905548,100.604803
2,2022-12-06 23:49:08,SNY20-1670370011-adhoc-0,3.1250,82.2500,208.8750,13.905196,100.604616
3,2022-12-06 23:49:09,SNY20-1670370011-adhoc-0,3.1875,82.8750,208.8125,13.904844,100.604429
4,2022-12-06 23:49:10,SNY20-1670370011-adhoc-0,3.2500,83.5000,208.7500,13.904492,100.604242
...,...,...,...,...,...,...,...
3963,2022-12-07 00:55:09,SNY20-1670370011-adhoc-0,3.5000,90.2500,208.2500,13.932567,100.620030
3964,2022-12-07 00:55:10,SNY20-1670370011-adhoc-0,3.3750,90.1875,208.1875,13.932196,100.619820
3965,2022-12-07 00:55:11,SNY20-1670370011-adhoc-0,3.2500,90.1250,208.1250,13.931824,100.619610
3966,2022-12-07 00:55:12,SNY20-1670370011-adhoc-0,3.1250,90.0625,208.0625,13.931452,100.619400


In [ ]:
test_dict['night']['1']

,timestamp,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
0,2022-12-06 23:49:06,SNY20-1670370011-adhoc-0,3.0000,81.0000,209.0000,13.905900,100.604990
1,2022-12-06 23:49:07,SNY20-1670370011-adhoc-0,3.0625,81.6250,208.9375,13.905548,100.604803
2,2022-12-06 23:49:08,SNY20-1670370011-adhoc-0,3.1250,82.2500,208.8750,13.905196,100.604616
3,2022-12-06 23:49:09,SNY20-1670370011-adhoc-0,3.1875,82.8750,208.8125,13.904844,100.604429
4,2022-12-06 23:49:10,SNY20-1670370011-adhoc-0,3.2500,83.5000,208.7500,13.904492,100.604242
...,...,...,...,...,...,...,...
3963,2022-12-07 00:55:09,SNY20-1670370011-adhoc-0,3.5000,90.2500,208.2500,13.932567,100.620030
3964,2022-12-07 00:55:10,SNY20-1670370011-adhoc-0,3.3750,90.1875,208.1875,13.932196,100.619820
3965,2022-12-07 00:55:11,SNY20-1670370011-adhoc-0,3.2500,90.1250,208.1250,13.931824,100.619610
3966,2022-12-07 00:55:12,SNY20-1670370011-adhoc-0,3.1250,90.0625,208.0625,13.931452,100.619400


In [ ]:
for item in Dict_flights:
    print(item)

NOK600-1670198400-schedule-0700
NOK340-1670198400-schedule-0684
SNY20-1670370011-adhoc-0
NOK122-1670197440-schedule-0484
AIQ3188-1670197440-schedule-0483
AIQ311-1670197440-schedule-0491
AIQ3029-1670197440-schedule-0464
AIQ3102-1670197440-schedule-0478
TLM702-1670196540-schedule-0499
NOK550-1670195580-schedule-0452
TLM600-1670195580-schedule-0482
TLM506-1670195580-schedule-0485


In [ ]:
Dict_flights[item]

KeyError: 'AIQ311-1670197440-schedule-0491'

In [ ]:
for time in test_dict:
    print(time)
    for flight in test_dict[time]:
        print(flight)

day
night
afk
1


In [ ]:
time

'night'

In [ ]:
Dict_flights['NOK340-1670198400-schedule-0684'][Dict_flights['NOK340-1670198400-schedule-0684']['altitude'] <= 80]

,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
2022-12-06 23:51:14,NOK340-1670198400-schedule-0684,3.000000,158.000000,209.000000,13.907960,100.602090
2022-12-06 23:51:15,NOK340-1670198400-schedule-0684,3.312500,158.125000,209.062500,13.907325,100.601720
2022-12-06 23:51:16,NOK340-1670198400-schedule-0684,3.625000,158.250000,209.125000,13.906690,100.601350
2022-12-06 23:51:17,NOK340-1670198400-schedule-0684,3.937500,158.375000,209.187500,13.906055,100.600980
2022-12-06 23:51:18,NOK340-1670198400-schedule-0684,4.250000,158.500000,209.250000,13.905420,100.600610
...,...,...,...,...,...,...
2022-12-07 00:32:24,NOK340-1670198400-schedule-0684,7.533333,130.866667,34.133333,15.213965,103.241958
2022-12-07 00:32:25,NOK340-1670198400-schedule-0684,7.400000,130.900000,34.100000,15.214456,103.242316
2022-12-07 00:32:26,NOK340-1670198400-schedule-0684,7.266667,130.933333,34.066667,15.214947,103.242674
2022-12-07 00:32:27,NOK340-1670198400-schedule-0684,7.133333,130.966667,34.033333,15.215439,103.243032


In [ ]:
# SNY20-1670370011-adhoc-0

In [ ]:
tmp = pd.read_csv('ONLY_TEST/AIQ359-1670457540-schedule-0563.csv')
tmp.head()

,Unnamed: 0,fa_flight_id,altitude,groundspeed,heading,latitude,longitude
0,2022-12-09 23:59:24,AIQ359-1670457540-schedule-0563,4.000,184.0000,208.0,13.898170,100.596750
1,2022-12-09 23:59:25,AIQ359-1670457540-schedule-0563,4.375,183.8125,208.0,13.897429,100.596352
2,2022-12-09 23:59:26,AIQ359-1670457540-schedule-0563,4.750,183.6250,208.0,13.896688,100.595955
3,2022-12-09 23:59:27,AIQ359-1670457540-schedule-0563,5.125,183.4375,208.0,13.895946,100.595557
4,2022-12-09 23:59:28,AIQ359-1670457540-schedule-0563,5.500,183.2500,208.0,13.895205,100.595160


In [ ]:
tmp = tmp[tmp['altitude'] <= 100]
tmp['check_index_1'] = tmp.index
tmp['check_index_2'] = tmp['check_index_1'].shift(-1)
tmp['drop'] = tmp['check_index_2']-tmp['check_index_1'] == 1

# tmp

In [ ]:
tmp.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            226, 227, 228, 229, 230, 231, 232, 233, 234, 235],
           dtype='int64', length=236)

In [ ]:
res = pd.DataFrame()
n = 0
while(tmp.iloc[n]['drop']) :
    res = res.append(tmp.iloc[n])
    n+=1
res = res.append(tmp.iloc[n])

res

,Unnamed: 0,fa_flight_id,altitude,groundspeed,heading,latitude,longitude,check_index_1,check_index_2,drop
0,2022-12-09 23:59:24,AIQ359-1670457540-schedule-0563,4.000000,184.000000,208.000000,13.898170,100.596750,0.0,1.0,1.0
1,2022-12-09 23:59:25,AIQ359-1670457540-schedule-0563,4.375000,183.812500,208.000000,13.897429,100.596352,1.0,2.0,1.0
2,2022-12-09 23:59:26,AIQ359-1670457540-schedule-0563,4.750000,183.625000,208.000000,13.896688,100.595955,2.0,3.0,1.0
3,2022-12-09 23:59:27,AIQ359-1670457540-schedule-0563,5.125000,183.437500,208.000000,13.895946,100.595557,3.0,4.0,1.0
4,2022-12-09 23:59:28,AIQ359-1670457540-schedule-0563,5.500000,183.250000,208.000000,13.895205,100.595160,4.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
231,2022-12-10 00:03:15,AIQ359-1670457540-schedule-0563,93.894737,302.526316,175.052632,13.731188,100.423133,231.0,232.0,1.0
232,2022-12-10 00:03:16,AIQ359-1670457540-schedule-0563,94.421053,302.894737,174.789474,13.729776,100.423277,232.0,233.0,1.0
233,2022-12-10 00:03:17,AIQ359-1670457540-schedule-0563,94.947368,303.263158,174.526316,13.728364,100.423422,233.0,234.0,1.0
234,2022-12-10 00:03:18,AIQ359-1670457540-schedule-0563,95.473684,303.631579,174.263158,13.726952,100.423566,234.0,235.0,1.0


In [ ]:
res.rename(columns={'Unnamed: 0':'timestamp'}, inplace=True)
res['timestamp'] = pd.to_datetime(res.timestamp)
res.set_index('timestamp',inplace=True)
res.index.inferred_type == "datetime64"

True

In [ ]:
res

,fa_flight_id,altitude,groundspeed,heading,latitude,longitude,check_index_1,check_index_2,drop
timestamp,,,,,,,,,
2022-12-09 23:59:24,AIQ359-1670457540-schedule-0563,4.000000,184.000000,208.000000,13.898170,100.596750,0.0,1.0,1.0
2022-12-09 23:59:25,AIQ359-1670457540-schedule-0563,4.375000,183.812500,208.000000,13.897429,100.596352,1.0,2.0,1.0
2022-12-09 23:59:26,AIQ359-1670457540-schedule-0563,4.750000,183.625000,208.000000,13.896688,100.595955,2.0,3.0,1.0
2022-12-09 23:59:27,AIQ359-1670457540-schedule-0563,5.125000,183.437500,208.000000,13.895946,100.595557,3.0,4.0,1.0
2022-12-09 23:59:28,AIQ359-1670457540-schedule-0563,5.500000,183.250000,208.000000,13.895205,100.595160,4.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...
2022-12-10 00:03:15,AIQ359-1670457540-schedule-0563,93.894737,302.526316,175.052632,13.731188,100.423133,231.0,232.0,1.0
2022-12-10 00:03:16,AIQ359-1670457540-schedule-0563,94.421053,302.894737,174.789474,13.729776,100.423277,232.0,233.0,1.0
2022-12-10 00:03:17,AIQ359-1670457540-schedule-0563,94.947368,303.263158,174.526316,13.728364,100.423422,233.0,234.0,1.0


In [ ]:
res.iloc[:1]

,fa_flight_id,altitude,groundspeed,heading,latitude,longitude,check_index_1,check_index_2,drop
timestamp,,,,,,,,,
2022-12-09 23:59:24,AIQ359-1670457540-schedule-0563,4.0,184.0,208.0,13.89817,100.59675,0.0,1.0,1.0


In [ ]:
if res.iloc[:1].between_time('07:00', '23:00').empty :
    print(1)

1


In [ ]:
res2 = res.reset_index()
res2

,timestamp,fa_flight_id,altitude,groundspeed,heading,latitude,longitude,check_index_1,check_index_2,drop
0,2022-12-09 23:59:24,AIQ359-1670457540-schedule-0563,4.000000,184.000000,208.000000,13.898170,100.596750,0.0,1.0,1.0
1,2022-12-09 23:59:25,AIQ359-1670457540-schedule-0563,4.375000,183.812500,208.000000,13.897429,100.596352,1.0,2.0,1.0
2,2022-12-09 23:59:26,AIQ359-1670457540-schedule-0563,4.750000,183.625000,208.000000,13.896688,100.595955,2.0,3.0,1.0
3,2022-12-09 23:59:27,AIQ359-1670457540-schedule-0563,5.125000,183.437500,208.000000,13.895946,100.595557,3.0,4.0,1.0
4,2022-12-09 23:59:28,AIQ359-1670457540-schedule-0563,5.500000,183.250000,208.000000,13.895205,100.595160,4.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
231,2022-12-10 00:03:15,AIQ359-1670457540-schedule-0563,93.894737,302.526316,175.052632,13.731188,100.423133,231.0,232.0,1.0
232,2022-12-10 00:03:16,AIQ359-1670457540-schedule-0563,94.421053,302.894737,174.789474,13.729776,100.423277,232.0,233.0,1.0
233,2022-12-10 00:03:17,AIQ359-1670457540-schedule-0563,94.947368,303.263158,174.526316,13.728364,100.423422,233.0,234.0,1.0
234,2022-12-10 00:03:18,AIQ359-1670457540-schedule-0563,95.473684,303.631579,174.263158,13.726952,100.423566,234.0,235.0,1.0


In [ ]:
print(res2.iloc[0].timestamp.date())

2022-12-09


In [ ]:
res2.timestamp.dt.date

0      2022-12-09
1      2022-12-09
2      2022-12-09
3      2022-12-09
4      2022-12-09
          ...    
231    2022-12-10
232    2022-12-10
233    2022-12-10
234    2022-12-10
235    2022-12-10
Name: timestamp, Length: 236, dtype: object

In [ ]:
type(tmp.loc[n])

pandas.core.series.Series

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
test_df = pd.read_csv(f'flights_data/AIQ642-1669939380-schedule-0476.csv')
test_df

,fa_flight_id,altitude,altitude_change,groundspeed,heading,latitude,longitude,timestamp,update_type
0,AIQ642-1669939380-schedule-0476,4,C,179,208,13.89944,100.59736,2022-12-03T23:57:52Z,A
1,AIQ642-1669939380-schedule-0476,10,C,184,208,13.88798,100.59111,2022-12-03T23:58:08Z,A
2,AIQ642-1669939380-schedule-0476,17,C,186,208,13.87573,100.58441,2022-12-03T23:58:24Z,A
3,AIQ642-1669939380-schedule-0476,23,C,190,233,13.86502,100.57491,2022-12-03T23:58:40Z,A
4,AIQ642-1669939380-schedule-0476,28,C,196,276,13.86214,100.56075,2022-12-03T23:58:56Z,A
...,...,...,...,...,...,...,...,...,...
164,AIQ642-1669939380-schedule-0476,31,D,191,15,21.25459,105.60944,2022-12-04T01:18:11Z,A
165,AIQ642-1669939380-schedule-0476,26,D,161,100,21.26655,105.64550,2022-12-04T01:19:00Z,A
166,AIQ642-1669939380-schedule-0476,24,D,158,105,21.26381,105.65796,2022-12-04T01:19:16Z,A
167,AIQ642-1669939380-schedule-0476,11,D,147,107,21.24337,105.72998,2022-12-04T01:20:56Z,A


In [6]:
# order_col = test_df.columns
# order_col

Index(['fa_flight_id', 'altitude', 'altitude_change', 'groundspeed', 'heading',
       'latitude', 'longitude', 'timestamp', 'update_type'],
      dtype='object')

In [5]:
# order_col = ['groundspeed', 'fa_flight_id', 'update_type']
# order_col = test_df.columns

# test_df[order_col]
# order_col

,groundspeed,fa_flight_id,update_type
0,179,AIQ642-1669939380-schedule-0476,A
1,184,AIQ642-1669939380-schedule-0476,A
2,186,AIQ642-1669939380-schedule-0476,A
3,190,AIQ642-1669939380-schedule-0476,A
4,196,AIQ642-1669939380-schedule-0476,A
...,...,...,...
164,191,AIQ642-1669939380-schedule-0476,A
165,161,AIQ642-1669939380-schedule-0476,A
166,158,AIQ642-1669939380-schedule-0476,A
167,147,AIQ642-1669939380-schedule-0476,A
